In [26]:
from pyarrow import Table
import pandas as pd
from pyarrow import flight
import json

from graphdatascience.query_runner.gds_arrow_client import AuthMiddleware, AuthFactory

In [72]:
location = flight.Location.for_grpc_tcp("localhost", 8491)

auth_middleware = AuthMiddleware(("neo4", "password"))   
client_options = {"middleware": [AuthFactory(auth_middleware)]}

flight_client = flight.FlightClient(location, **client_options)

# Projection

In [73]:
result = flight_client.do_action(
    flight.Action(
        "V1/CREATE_GRAPH_FROM_TRIPLETS",
        json.dumps(
            {
                "name": "g"
            }
        ).encode("utf-8")
    )    
)

collected_result = list(result)
assert len(collected_result) == 1

importJobId = json.loads(collected_result[0].body.to_pybytes().decode())

In [74]:
data = [
    {"sourceNode": 0, "targetNode": 1},
    {"sourceNode": 0, "targetNode": 2},
    {"sourceNode": 1, "targetNode": 3},
    {"sourceNode": 4, "targetNode": 5},
]

flight_descriptor = {
    "version": "v1",
    "name": "TRIPLETS",
    "body": {
        "jobId": importJobId,
    }
}

table = Table.from_pandas(pd.DataFrame(data))

writer, ackStream = flight_client.do_put(
    flight.FlightDescriptor.for_command(json.dumps(flight_descriptor).encode("utf-8")),
    table.schema    
)

with writer:
    writer.write_table(table)
    ackStream.read()

In [ ]:
result = flight_client.do_action(
    flight.Action(
        "V1/TRIPLET_LOAD_DONE",
        json.dumps(
            {
                "jobid": importJobId
            }
        ).encode("utf-8")
    )    
)

collected_result = list(result)
assert len(collected_result) == 1

json.loads(collected_result[0].body.to_pybytes().decode())

# Run WCC

In [79]:
result = flight_client.do_action(
    flight.Action(
        "V1/wcc",
        json.dumps(
            {
                "mutateProperty": "communityId"
            }
        ).encode("utf-8")
    )    
)

In [80]:
collected_result = list(result)
assert len(collected_result) == 1

collected_result[0].body.to_pybytes().decode()

'441e4e7d-beac-47e6-8226-d2809bc455bb'